# Data Preperation

This notebook contains the data exploration and preperation.

In [146]:
import os
import pandas as pd
import numpy as np

In [147]:
players_df = pd.read_csv('C:\mahmoud uni\TU\SS2024\KGs\Portfolio\dataset\player_dataset.csv')
teams_df = pd.read_csv('C:\mahmoud uni\TU\SS2024\KGs\Portfolio\dataset\\team_dataset.csv')

C:\Users\mabsa\AppData\Local\Temp\ipykernel_4556\1980341846.py:1: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  players_df = pd.read_csv('C:\mahmoud uni\TU\SS2024\KGs\Portfolio\dataset\player_dataset.csv')


## Data Exploration

In [148]:
# print("player dataframe columns: ", players_df.columns.to_list())
# print("team dataframe columns: ", players_df.columns.to_list())


### Player File

In [149]:
# Basic info
print(players_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180021 entries, 0 to 180020
Columns: 109 entries, player_id to gk
dtypes: float64(20), int64(43), object(46)
memory usage: 149.7+ MB
None


In [150]:
# Summary statistics
#print(players_df.describe())

In [151]:
missing_values = players_df.isnull().sum()  # Calculate the number of missing values per column
missing_values = missing_values[missing_values > 0]  # Filter columns with missing values
#print(missing_values)
# Calculate the fraction of missing values
fraction_missing = missing_values / len(players_df) * 100
print(fraction_missing)


value_eur                          1.195972
wage_eur                           1.026547
club_team_id                       1.035990
club_name                          1.035990
league_id                          1.035990
league_name                        1.035990
league_level                       1.249854
club_position                      1.035990
club_jersey_number                 1.035990
club_loaned_from                  94.043473
club_joined_date                   6.992518
club_contract_valid_until_year     1.035990
nation_team_id                    94.390654
nation_position                   94.390654
nation_jersey_number              94.390654
release_clause_eur                32.940046
player_tags                       92.268680
player_traits                     54.558079
pace                              11.123147
shooting                          11.123147
passing                           11.123147
dribbling                         11.123147
defending                       

Wir gehen mal die Columns durch und schauen welche Variablen wir rausschmeißen und welche wir manipulierne bevor wir uns um die NAs kümmern

- Club_position will be changed to whether player is a starter or not.
- player_position will be changed so that its just GK, DEF, MF, Attack
- Club loaned from -> on loan


work rate aufteilen in workrate off und workrate def
Wir werfen die positions spezifischen Ratings weg da die relevante information sich im allgemeinen Rating sowie in den einzelnen stats befindet. Wir lassen die Nationalmannschafts daten weg, weil fifa nicht alle nationalitäten beinhaltet und somit information verloren gehen würde 
Release Clause wird gedroppt weil diese daten nicht der realität entsprechen

In [152]:
# We will drop columns not needed for the analysis
drop_cols = ['fifa_update', 'update_as_of', 'short_name', 'club_jersey_number', 'nation_team_id', 
            'nation_position', 'dob', 'nation_jersey_number', 'club_loaned_from', 'player_tags',
            'player_traits', 'real_face', "release_clause_eur",'ls', 'st', 'rs', 'lw', 'lf',
            'cf', 'rf', 'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm',  'rcm', 'rm', 'lwb', 'ldm', 
            'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk']

players_clean = players_df.drop(columns=drop_cols)

In [153]:
## On loan
# Create the on_loan column
players_clean['on_loan'] = players_df['club_loaned_from'].apply(lambda x: 0 if pd.isnull(x) else 1)


## Position
# Define conditions
conditions = [
    players_df['club_position'] == "RES",
    players_df['club_position'] == "SUB",
    players_df['club_position'].notnull()  # Covers all other non-null cases
]
# Define corresponding values
values = [0, 1, 2]
players_clean['club_position'] = np.select(conditions, values, default=np.nan)

# transform into date
players_clean['club_joined_date'] = pd.to_datetime(players_clean["club_joined_date"])


In [154]:
# Define a mapping function
def map_position(position):
    if not isinstance(position, str):  # Handle missing or invalid values
        return np.nan
    first_position = position.split(', ')[0]  # Get the first position
    if first_position == 'GK':
        return 'GK'
    elif first_position in ['ST', 'CF']:
        return 'ATT'
    elif first_position in ['RW', 'LW', 'RWB', 'LWB']:
        return 'WING'
    elif first_position in ['CM', 'CAM', 'CDM', 'LM', 'RM']:
        return 'MID'
    elif first_position in ['CB', 'RB', 'LB']:
        return 'DEF'
    else:
        return np.nan

# Apply the function to create a new column
players_clean['position_category'] = players_clean['player_positions'].apply(map_position)

#print(players_clean[['player_positions', 'position_category']].head())


#### NA Handeling

Für die GK stats kontrollieren wir ob das Torhüter sind und imputieren nicht GK 0.
1 GK hat NA den nehmen wir raus

In [155]:

# remove gk with missing gk speed
players_clean = players_clean.drop(index=69892) # data wrangler
players_clean.reset_index(drop=True, inplace=True)

players_clean['goalkeeping_speed'] = players_clean['goalkeeping_speed'].fillna(0)


In [156]:
temp = players_clean[['long_name', 'position_category', 'goalkeeping_speed']]


In [157]:
missing_values = players_clean.isnull().sum()  # Calculate the number of missing values per column
missing_values = missing_values[missing_values > 0]  # Filter columns with missing values
#print(missing_values)
# Calculate the fraction of missing values
fraction_missing = missing_values / len(players_df) * 100
print(fraction_missing)

value_eur                          1.195972
wage_eur                           1.026547
club_team_id                       1.035990
club_name                          1.035990
league_id                          1.035990
league_name                        1.035990
league_level                       1.249854
club_position                      1.035990
club_joined_date                   6.992518
club_contract_valid_until_year     1.035990
pace                              11.123147
shooting                          11.123147
passing                           11.123147
dribbling                         11.123147
defending                         11.123147
physic                            11.123147
mentality_composure               18.268980
dtype: float64


In [ ]:
# Group by 'league_name' and transform to get the most common 'league_level'
league_level_fill = players_clean.groupby('league_name', )['league_level'].transform(lambda x: x.mode()[0] if not x.mode().empty else np.nan)
players_clean['league_level'] = players_clean['league_level'].fillna(league_level_fill)



TODO:
 - check mal spalten wie league_level, macht das Sinn?
 - check mal ähnliche spalten auf consistency
 - NA handling müssen wir fertig machen

In [ ]:

grouped_avg_value = players_clean.groupby(['age', 'overall', 'league_level', 'position_category'])['value_eur'].transform('mean')

# Fill missing values in 'value_eur' with the calculated group averages
players_clean['value_eur'] = players_clean['value_eur'].fillna(grouped_avg_value) # remaining NAs are players over 40
players_clean['value_eur'] = players_clean['value_eur'].fillna(0)



In [144]:
# now we check whether the NAs overlap
# Select rows with at least one NaN
rows_with_nan = players_clean[players_clean.isnull().any(axis=1)]
players_clean['nans_per_row'] = players_clean.isnull().sum(axis=1)

# Remove rows with more than 10 missing values
players_clean = players_clean[players_clean.isnull().sum(axis=1) <= 7]

# Reset index if needed
players_clean.reset_index(drop=True, inplace=True)

# Check the updated DataFrame
print(players_clean.shape)

(177997, 72)


In [145]:
missing_values = players_clean.isnull().sum()  # Calculate the number of missing values per column
missing_values = missing_values[missing_values > 0]  # Filter columns with missing values
#print(missing_values)
# Calculate the fraction of missing values
fraction_missing = missing_values / len(players_df) * 100
print(fraction_missing)


club_joined_date        5.868760
pace                   10.877064
shooting               10.877064
passing                10.877064
dribbling              10.877064
defending              10.877064
physic                 10.877064
mentality_composure    17.919576
dtype: float64


### Team File

In [6]:
teams_df.league_name.unique()

array(['Premier League', 'La Liga', 'Friendly International',
       'Bundesliga', 'Ligue 1', 'Serie A', 'Liga Portugal', 'Super Lig',
       'Pro League', 'Eredivisie', 'Liga Profesional', 'Championship',
       'Super League', 'Premiership', 'Jupiler Pro League', 'La Liga 2',
       'Fortuna Liga', 'Major League Soccer', 'Superliga',
       '2. Bundesliga', '1. HNL', 'Ligue 2', 'NB I.', 'Liga BetPlay',
       'Liga Pro', 'Primera División', 'Serie B', 'Allsvenskan',
       'Eliteserien', 'K League 1', 'Primera Division', 'Ekstraklasa',
       '1. Division', 'Liga 1', 'League One', 'Liga De Futbol Prof',
       '3. Liga', 'A-League', 'League Two', 'Veikkausliiga',
       'Premier Division', 'Liga MX', 'J-League'], dtype=object)

In [ ]:
print(teams_df.info())
print(teams_df.describe())


In [ ]:
# Check for missing values
print(teams_df.isnull().sum())

NationalT Teams raus